<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In this notebook, I will explain how to run an election on your voting profile

In [2]:
import numpy as np
import embedded_voting as ev

Let's say we have 5 candidates and 3 groups of voters :
   * The red group contains 50% of the voters, and the average scores of candidates given by this group are [0.9,0.7,0.5,0.3,0.1]
   * The green group contains 25% of the voters, and the average scores of candidates given by this group are [0.1,0.3,0.5,0.7,0.8]
   * The blue group contains 25% of the voters, and the average scores of candidates given by this group are [0.1,0.3,0.5,0.7,0.8]